In [5]:
library(ggplot2)
library(readxl)
library(dplyr)
url <- "http://www.anfavea.com.br/docs/SeriesTemporais_Autoveiculos.xlsm"
filename <- "Gabriel André/Estudo R/Veiculos.xslm"
download.file(url = url, destfile = filename, mode = "wb")
data <- read_excel("Gabriel André/Estudo R/Veiculos.xslm", skip = 4)
file.remove("Gabriel André/Estudo R/Veiculos.xslm")
data <- data[,-seq(7,26,1)]
colnames(data) <- c("Data", "Licenciamento_Tot","Licenciamento_Nac",
                    "Licenciamento_Import",
                    "Produção", "Exportação")
data[,"%_produção_exportada"] <- (data$Exportação/data$Produção)*100
data[,"Crescimento_Producao"] <- matrix(0,nrow = nrow(data), ncol = 1)
for(i in 13:nrow(data)){
  data$Crescimento_Producao[i] <- (data$Produção[i]/data$Produção[i-12]-1)
}


Attaching package: 'dplyr'


The following objects are masked from 'package:data.table':

    between, first, last


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




ERROR: Error in download.file(url = url, destfile = filename, mode = "wb"): não foi possível abrir o arquivo de destino 'Gabriel André/Estudo R/Veiculos.xslm', motivo 'No such file or directory'
